<a href="https://colab.research.google.com/github/alexx99gg/ADDL/blob/main/TFG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Read_data

In [12]:
!pip install pandas_plink

In [13]:
import math

import numpy as np
import pandas


def read_diagnose(file_path: str = '../diagnosis_data/DXSUM_PDXCONV_ADNIALL.csv', verbose=False):
    # Read diagnostic summary
    diagnostic_summary = pandas.read_csv(file_path, index_col='PTID')
    diagnostic_summary_headers = diagnostic_summary.columns.tolist()

    # Create dictionary
    diagnostic_dict: dict = {}
    for key, data in diagnostic_summary.iterrows():
        # Iterate for each row of the document
        phase: str = data[diagnostic_summary_headers.index('Phase')]
        diagnosis: float = -1.
        if phase == "ADNI1":
            diagnosis = data[diagnostic_summary_headers.index('DXCURREN')]
        elif phase == "ADNI2" or phase == "ADNIGO":
            dxchange = data[diagnostic_summary_headers.index('DXCHANGE')]
            if dxchange == 1 or dxchange == 7 or dxchange == 9:
                diagnosis = 1.
            if dxchange == 2 or dxchange == 4 or dxchange == 8:
                diagnosis = 2.
            if dxchange == 3 or dxchange == 5 or dxchange == 6:
                diagnosis = 3.
        elif phase == "ADNI3":
            diagnosis = data[diagnostic_summary_headers.index('DIAGNOSIS')]
        else:
            print(f"ERROR: Not recognized study phase {phase}")
            exit(1)
        # Update dictionary
        if not math.isnan(diagnosis):
            diagnostic_dict[key] = diagnosis
    if verbose:
        print_diagnostic_dict_summary(diagnostic_dict)
    return diagnostic_dict


def print_diagnostic_dict_summary(diagnostic_dict: dict):
    print(f"Number of diagnosed patients: {len(diagnostic_dict.items())}\n")
    n_NL = 0
    n_MCI = 0
    n_AD = 0
    for (key, data) in diagnostic_dict.items():
        if data == 1:
            n_NL += 1
        if data == 2:
            n_MCI += 1
        if data == 3:
            n_AD += 1
    print(f"Number of NL patients: {n_NL}\n"
          f"Number of MCI patients: {n_MCI}\n"
          f"Number of AD patients: {n_AD}\n")


def generate_dataset(diagnostic_dict: dict, fam, bed):
    n_wgs_samples = bed.shape[1]
    n_snps = bed.shape[0]
    y = []
    # Generate label data
    # Only difference between Alzheimer and not alzheimer
    for test in range(n_wgs_samples):
        # Read iid from wgs data
        iid = fam.iat[test, 1]
        # Get diagnose corresponding to the iid
        last_diagnose = diagnostic_dict[iid]
        has_alzheimer = False
        if last_diagnose == 1 or last_diagnose == 2:
            # Mild cognitive impairment simply considered as not alzheimer
            has_alzheimer = False
        elif last_diagnose == 3:
            has_alzheimer = True
        else:
            print("Error: diagnosis not recognized")
        y.append(has_alzheimer)

    y = np.asarray(y)
    y = y.reshape((n_wgs_samples, 1))

    # Generate features data
    # Replace NaN values (missing genotype)
    bed = np.nan_to_num(bed, 1)
    x = np.asarray(bed)
    x = x.transpose((1, 0))

    # Normalize dataset values [0,1]
    x = (x - np.min(x)) / np.ptp(x)
    return x, y


# Train_data

In [14]:
import tensorflow as tf
from tensorflow.keras import initializers
from tensorflow.keras import layers


def create_MLP_model(n_SNPs: int):
    # Define Neural Network architecture

    model = tf.keras.models.Sequential()
    model.add(layers.Flatten(input_shape=(n_SNPs, 1)))

    # Dense with neurons equal to the number of inputs SNPs, ReLU as activation, L2 Regularization and He Initialization
    model.add(
        layers.Dense(min(n_SNPs, 2 ** 16), activation='relu', kernel_regularizer='l2', kernel_initializer=initializers.he_normal))
    # Batch Normalization, Dropout Layer with 30% of inputs to drop,Gaussian Noise with 0.3 as Standard Deviation
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.2))
    model.add(layers.GaussianNoise(0.3))

    # Dense Layer with 1024 outputs, ReLU as activation, L2 Regularization and He Initialization
    model.add(
        layers.Dense(2 ** 10, activation='relu', kernel_regularizer='l2', kernel_initializer=initializers.he_normal))
    # Batch Normalization, Dropout Layer with 30% of inputs to drop
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.25))

    # Dense Layer with 512 outputs, ReLU as activation, L2 Regularization and He Initialization
    model.add(
        layers.Dense(2 ** 9, activation='relu', kernel_regularizer='l2', kernel_initializer=initializers.he_normal))
    # Batch Normalization, Dropout Layer with 30% of inputs to drop
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.25))
    

    # Dense Layer with 256 outputs, ReLU as activation, L2 Regularization and He Initialization
    model.add(
        layers.Dense(2 ** 8, activation='relu', kernel_regularizer='l2', kernel_initializer=initializers.he_normal))
    # Batch Normalization, Dropout Layer with 30% of inputs to drop
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.25))

    # Dense Layer with 128 outputs, ReLU as activation, L2 Regularization and He Initialization
    model.add(
        layers.Dense(2 ** 7, activation='relu', kernel_regularizer='l2', kernel_initializer=initializers.he_normal))
    # Batch Normalization, Dropout Layer with 30% of inputs to drop
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.25))

    # Dense Layer with 64 outputs, ReLU as activation, L2 Regularization and He Initialization
    model.add(
        layers.Dense(2 ** 6, activation='relu', kernel_regularizer='l2', kernel_initializer=initializers.he_normal))
    # Batch Normalization, Dropout Layer with 30% of inputs to drop
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.25))

    # OUTPUT
    # Dense with 2 outputs, sigmoid activation
    model.add(layers.Dense(1, activation='sigmoid'))

    # Compile the Neural Network
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy', 'AUC']
    )

    return model


# Plot_utils

In [15]:
import matplotlib.pyplot as plt
import numpy as np


def plot_training_history(history):
    plt.plot(history.history['accuracy'], label='train')
    plt.plot(history.history['val_accuracy'], label='test')
    plt.legend()
    plt.show()


def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr)
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')

    # Draw straight line
    x = np.linspace(0, 1, 100)
    y = x
    plt.plot(x, y, '--r')
    plt.show()


# main

In [16]:
from pandas_plink import read_plink
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.callbacks import EarlyStopping

root_folder = '/content/drive/MyDrive/TFG/'

diagnostic_dict = read_diagnose(file_path= root_folder + 'diagnosis_data/DXSUM_PDXCONV_ADNIALL.csv')

(bim, fam, bed) = read_plink(root_folder + '/wgs_data/clean')

# bed:
#   0 -> First allele
#   1 -> Heterozygous
#   2 -> Second allele
#   math.nan -> missing genotype

n_wgs_samples = bed.shape[1]
n_SNPs = bed.shape[0]

print(f"Current number of WGS samples: {n_wgs_samples}\n")
print(f"Current number of variants per WGS sample: {n_SNPs}\n")

# Generate dataset from input data
x, y = generate_dataset(diagnostic_dict, fam, bed)
# Split data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, shuffle=True)
print(f"Shape of train data: {x_train.shape}")
print(f"Shape of train labels: {y_train.shape}")

print(f"Shape of test data: {x_test.shape}")
print(f"Shape of test labels: {y_test.shape}")
# print(y_test)

# Create and fit model
print("Creating model...")
model = create_MLP_model(n_SNPs)
print("Creating model... DONE")

print("Training model...")
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=100, verbose=1)
history = model.fit(x_train, y_train, epochs=20, validation_split=0.3, callbacks=[es])  # validation_split=0.3
print("Training model... DONE")

plot_training_history(history)

print("Evaluate model...")
model.evaluate(x_test, y_test, verbose=2)

y_test_prob = model.predict(x_test)


fpr, tpr, thresholds = roc_curve(y_test, y_test_prob)
plot_roc_curve(fpr, tpr)

auc_score = roc_auc_score(y_test, y_test_prob)


Mapping files: 100%|██████████| 3/3 [00:00<00:00,  8.13it/s]


Current number of WGS samples: 1550

Current number of variants per WGS sample: 125005

Shape of train data: (1038, 125005)
Shape of train labels: (1038, 1)
Shape of test data: (512, 125005)
Shape of test labels: (512, 1)
Creating model...


ResourceExhaustedError: ignored